# Ejercicio 2 historico RV Alphavantage

*Consigue el histórico de renta variable, minuto a minuto, de 5 días, de las siguientes empresas*

Recomendación: Alphavantage / Tiempo objetivo: 30 minutos

In [ ]:
!pip install alpha_vantage

In [3]:
import pandas as pd
import requests
import alpha_vantage
import json
import datetime
from pandas.io.json import json_normalize
from alpha_vantage.timeseries import TimeSeries
import pprint
from datetime import datetime, date, timedelta
import numpy as np
import time

In [4]:
activos_a_descargar=("AAPL","AXP","BA","CAT","CSCO","CVX","DIS","GE","GS","HD","IBM","INTC","JNJ","JPM")

In [5]:
len(activos_a_descargar)

14

Hacemos la llamada a Alphavantage

In [6]:
lista_activos=[]
key='AOURS93CWW9EDZWO'

for i in range(len(activos_a_descargar)):
    
    activo=activos_a_descargar[i]
    
    #Hacemos la llamada a Alphavantage
    ts = TimeSeries(key=key, output_format='pandas')
    df=data, meta_data = ts.get_intraday(symbol=activos_a_descargar[i],interval='1min', outputsize='full')
    df= df[0]
    df['systime']=pd.to_datetime(pd.Timestamp.now(), format='%Y/%m/%d %H:%M:%S')
    
    # Modificamos la hora que consta en los datos con la diferencia que acabamos de calcular.
    df['diferencia']=(df.systime-df.index)/np.timedelta64(1,'h')
    df['diferencia']=np.floor(df.diferencia)
    df['diferencia']=df.diferencia*(np.timedelta64(1,'h'))
    df.index=df.index+df.diferencia
    df=df.drop(['diferencia'],1)
    globals()[activo]=df.copy()# lo llevamos a un dataframe con el nombre del activo
    
    # Guardamos los datos con el nombre del activo en una lista.    
    lista_activos.append(df)

    # Solo nos dejan 5 llamadas por minuto. Debemos tener esto en cuenta a la hora de hacer la llamada.
    if activos_a_descargar.index(activos_a_descargar[i]) % 5 == 0:
        time.sleep(62)
        
    print((activos_a_descargar[i]))

AAPL
AXP
BA
CAT
CSCO
CVX
DIS
GE
GS
HD
IBM
INTC
JNJ
JPM


In [8]:
AAPL.head()

,1. open,2. high,3. low,4. close,5. volume,systime
2020-06-30 16:30:00,351.6250,351.7800,351.20,351.420,68179.0,2020-06-30 16:41:09.500026
2020-06-30 16:31:00,351.4200,351.4900,350.99,351.230,93942.0,2020-06-30 16:41:09.500026
2020-06-30 16:32:00,351.2009,351.7200,351.20,351.520,69117.0,2020-06-30 16:41:09.500026
2020-06-30 16:33:00,351.5100,351.5100,351.21,351.335,43588.0,2020-06-30 16:41:09.500026
2020-06-30 16:34:00,351.3300,351.4427,350.32,351.100,201642.0,2020-06-30 16:41:09.500026


### Otra manera de hacerlo (usando zonas horarias)

In [9]:
from alpha_vantage.timeseries import TimeSeries
import time
import pandas as pd
import pytz

In [10]:
tickers = ["AAPL","AXP","BA","CAT","CSCO","CVX","DIS","GE","GS","HD","IBM","INTC","JNJ","JPM"]

myDic_alpha = dict()
data = []

namesList = ['open', 'high', 'low', 'close', 'volume']
ts = TimeSeries(key='AOURS93CWW9EDZWO', output_format='pandas')
us_timezone = pytz.timezone('US/Eastern')
local_timezone = pytz.timezone('Europe/London')

for i in range(0, len(tickers)):
    data, meta_data = ts.get_intraday(symbol=tickers[i], interval='1min', outputsize='full')
    data.columns = namesList
    data.index = data.index.tz_localize(us_timezone).tz_convert(local_timezone)
    myDic_alpha[tickers[i]] = pd.DataFrame(data)
    time.sleep(13)

In [11]:
myDic_alpha.keys()

dict_keys(['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM'])

In [14]:
myDic_alpha["AAPL"][1:10]

,open,high,low,close,volume
date,,,,,
2020-06-18 15:31:00+01:00,351.4200,351.4900,350.9900,351.2300,93942.0
2020-06-18 15:32:00+01:00,351.2009,351.7200,351.2000,351.5200,69117.0
2020-06-18 15:33:00+01:00,351.5100,351.5100,351.2100,351.3350,43588.0
2020-06-18 15:34:00+01:00,351.3300,351.4427,350.3200,351.1000,201642.0
2020-06-18 15:35:00+01:00,351.0791,352.0000,351.0791,351.8300,132031.0
2020-06-18 15:36:00+01:00,351.8500,352.5100,351.6700,352.1400,132825.0
2020-06-18 15:37:00+01:00,352.1000,352.1400,351.8539,351.9600,68868.0
2020-06-18 15:38:00+01:00,351.9600,351.9600,351.6800,351.6843,55269.0
2020-06-18 15:39:00+01:00,351.7100,352.0100,351.7100,351.8976,47467.0
